In [1]:
#nans set to 0

# Do I want to make the padding value 0 so i ignore all rows with 0??
# should i use a different optimizer for my compiler?? maybe adam

from google.colab import drive
drive.mount('/content/drive/')

In [2]:
import pandas as pd
import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
ROOT_DATA_DIR = '/content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs'
#ROOT_DATA_DIR = '../raw_data/Out_Feature_CSVs'
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]

def get_y(data_path, class_list):
    
    data_dict = {"csv_paths":[], "csv_files":[], "labels":[]}
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_csvs = [l for l in os.listdir(dir_path) if l.split(".")[-1]=="csv"]
        print(f"Found {len(list_csvs)} CSVs in {dir_path} Directory")
        data_dict["csv_files"].extend(list_csvs)
        data_dict["labels"].extend([i for k in range(len(list_csvs))])
        data_dict["csv_paths"].extend([dir_path for _ in range(len(list_csvs))])
    total_csvs = len(data_dict["csv_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of csvs : {total_csvs}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST))


test_data = pd.DataFrame(get_y(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST))  

Found 45 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Bad Directory
Found 45 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Train/Good Directory
Number of csvs : 90
Number of Labels : 90
Found 13 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Bad Directory
Found 13 CSVs in /content/drive/MyDrive/Squat_Pro/Out_Feature_CSVs/Test/Good Directory
Number of csvs : 26
Number of Labels : 26


In [4]:
train_data.head()

,csv_paths,csv_files,labels
0,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9362_02.csv,0
1,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,65.csv,0
2,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9362_03.csv,0
3,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,IMG_9341_02.csv,0
4,/content/drive/MyDrive/Squat_Pro/Out_Feature_C...,47.csv,0


In [5]:
#create y_train and y_test as float64 type
y_train = train_data['labels']/1.0
y_test = test_data['labels']/1.0

In [6]:
def csv_to_list_of_lists(csv_path):
    df = pd.read_csv(csv_path)
    df.fillna(0, inplace=True)
    list_of_lists = df.iloc[:,1:].values.tolist()
    return list_of_lists

def get_x(dataframe):
  list_of_csv_files = [os.path.join(row['csv_paths'], row["csv_files"]) for index, row in dataframe.iterrows()]
  X_list = [csv_to_list_of_lists(f) for f in list_of_csv_files]
  return X_list

In [7]:
#create X_train and X_test
X_train = get_x(train_data)
X_test = get_x(test_data)

In [8]:
#pad X_train so all arrays are of the same shape
X_train_pad = pad_sequences(X_train, dtype='float32', padding='post', value=-1000)
print(X_train_pad.shape)

#pad X_test so all arrays are of the same shape
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post', value=-1000, maxlen=X_train_pad.shape[1])
print(X_test_pad.shape)

(90, 169, 2048)
(26, 169, 2048)


#LSTM_Model5 - Learning Rate and many layers
#New
#1
#[0.6656529307365417, 0.5444444417953491]
#[0.7044618129730225, 0.5]
#2
#[0.8229615688323975, 0.5444444417953491]
#[0.8788918256759644, 0.5]
#3
#
#

#Old
#1
#[0.7743281126022339, 0.6499999761581421]
#[0.7172174453735352, 0.6499999761581421]
#2
#[0.814126193523407, 0.5375000238418579]
#[0.8420926928520203, 0.5]
#3
#[0.5910772085189819, 0.6499999761581421]
#[0.5311039686203003, 0.699999988079071]

model_LSTM = Sequential()

model_LSTM.add(layers.Masking(mask_value=-1000))

model_LSTM.add(layers.LSTM(200, activation='tanh', return_sequences=True))
model_LSTM.add(layers.LSTM(150, activation='tanh'))
model_LSTM.add(layers.Dense(125, activation='relu'))
model_LSTM.add(layers.Dense(100, activation='relu'))
model_LSTM.add(layers.Dense(75, activation='relu'))
model_LSTM.add(layers.Dense(60, activation='relu'))
model_LSTM.add(layers.Dense(40, activation='relu'))
model_LSTM.add(layers.Dense(25, activation='relu'))    
model_LSTM.add(layers.Dense(10, activation='relu'))
model_LSTM.add(layers.Dense(1, activation='sigmoid'))

opt = optimizers.RMSprop(learning_rate=0.003)

model_LSTM.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

In [9]:
#LSTM_Model_March_8 (Binary_accuracy)
#1
#[0.5378538966178894, 0.9666666388511658]
#[4.459084510803223, 0.692307710647583]

def build_model(input_shape):

  model_LSTM = Sequential()

  model_LSTM.add(layers.Masking(mask_value=-1000, input_shape=input_shape))
  model_LSTM.add(layers.LSTM(64, activation='tanh', return_sequences=True))
  model_LSTM.add(layers.Dropout(0.2))
  model_LSTM.add(layers.LSTM(32, activation='tanh'))
  model_LSTM.add(layers.Dense(8, activation='relu'))
  model_LSTM.add(layers.Dense(4, activation='relu'))
  model_LSTM.add(layers.Dense(1, activation='sigmoid'))

  model_LSTM.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=[tf.keras.metrics.BinaryAccuracy()])
  return model_LSTM

  print(model_LSTM.summary())

model = build_model(input_shape=(X_train_pad.shape[1], X_train_pad.shape[2]))

#Fitting the model to the train set
es = EarlyStopping(patience=15)

model_LSTM.fit(X_train_pad, y_train, 
              epochs=175, 
              batch_size=32, 
              verbose=1, 
              callbacks = [es],
              validation_split=0.2,
              shuffle=True)

In [10]:
#Fitting the model to the train set
es = EarlyStopping(patience=50)

model.fit(X_train_pad, y_train, 
          epochs=1750, 
          batch_size=8, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,
          shuffle=True)

Epoch 1/1750
9/9 [==============================] - 13s 755ms/step - loss: 0.6942 - binary_accuracy: 0.5425 - val_loss: 0.8252 - val_binary_accuracy: 0.1667
Epoch 2/1750
9/9 [==============================] - 3s 345ms/step - loss: 0.6297 - binary_accuracy: 0.7278 - val_loss: 0.8977 - val_binary_accuracy: 0.0556
Epoch 3/1750
9/9 [==============================] - 3s 330ms/step - loss: 0.6829 - binary_accuracy: 0.5872 - val_loss: 0.8418 - val_binary_accuracy: 0.2222
Epoch 4/1750
9/9 [==============================] - 3s 336ms/step - loss: 0.6761 - binary_accuracy: 0.6006 - val_loss: 0.8931 - val_binary_accuracy: 0.1667
Epoch 5/1750
9/9 [==============================] - 3s 334ms/step - loss: 0.6528 - binary_accuracy: 0.6539 - val_loss: 1.0537 - val_binary_accuracy: 0.0000e+00
Epoch 6/1750
9/9 [==============================] - 3s 344ms/step - loss: 0.6208 - binary_accuracy: 0.6931 - val_loss: 0.9474 - val_binary_accuracy: 0.1667
Epoch 7/1750
9/9 [==============================] - 3s 339m

In [16]:
model.save("my_model")
model.save_weights("weights.h5")

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [11]:
#Evaluate the model on the train set
model.evaluate(X_train_pad, y_train)

3/3 [==============================] - 1s 242ms/step - loss: 0.5379 - binary_accuracy: 0.9667


[0.5378538966178894, 0.9666666388511658]

In [12]:
#Evaluate the model on the test set
model.evaluate(X_test_pad, y_test)

1/1 [==============================] - 0s 254ms/step - loss: 4.4591 - binary_accuracy: 0.6923


[4.459084510803223, 0.692307710647583]

In [13]:
#predictions of the test set compared to the actuals
model.predict(X_test_pad)

array([[1.23167583e-08],
       [1.19598962e-08],
       [1.18199273e-08],
       [1.29870474e-02],
       [2.50020623e-03],
       [1.00000000e+00],
       [1.19802159e-08],
       [9.96825402e-05],
       [1.00000000e+00],
       [1.36460265e-08],
       [1.00000000e+00],
       [1.18512382e-08],
       [1.18976908e-08],
       [1.00000000e+00],
       [9.36895609e-04],
       [1.36487024e-05],
       [9.99999881e-01],
       [1.72730438e-06],
       [1.00000000e+00],
       [4.81068128e-08],
       [1.00000000e+00],
       [9.99999642e-01],
       [9.99959350e-01],
       [1.00000000e+00],
       [2.03799868e-06],
       [9.99999583e-01]], dtype=float32)

In [14]:
y_test

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    1.0
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
Name: labels, dtype: float64